In [28]:
import pytest
import numpy as np
from statsmodels.datasets import grunfeld
import time
from datetime import datetime

from ipca import InstrumentedPCA


# Test Construction Errors
def test_construction_errors():
  with pytest.raises(ValueError):
    InstrumentedPCA(n_factors=0)
  with pytest.raises(NotImplementedError):
    InstrumentedPCA(intercept='jabberwocky')
  with pytest.raises(ValueError):
    InstrumentedPCA(iter_tol=2)


# Create test data and run package
data = grunfeld.load_pandas().data
data.year = data.year.astype(np.int64)

#data.firm = data.firm.apply(lambda x: x.decode('utf-8'))
# Establish unique IDs to conform with package
N = len(np.unique(data.firm))
ID = dict(zip(np.unique(data.firm).tolist(), np.arange(1, N+1)+5))
data.firm = data.firm.apply(lambda x: ID[x])
# Ensure that ordering of the data is correct
data = data[['firm', 'year', 'invest', 'value', 'capital']]

# prep PSF test vars test vars
PSF1 = np.random.randn(len(np.unique(data.loc[:, 'year'])), 1)
PSF1 = PSF1.reshape((1, -1))
PSF2 = np.random.randn(len(np.unique(data.loc[:, 'year'])), 2)
PSF2 = PSF2.reshape((2, -1))
data = data.set_index(['firm', 'year'])
for i in set(i[0] for i in data.index):
    data.loc[(i,1935),'invest'] = np.nan
data_y = data['invest']
data_x = data.drop('invest', axis=1)

t0 = datetime.now()

# Test InstrumentedPCA
regr = InstrumentedPCA(n_factors=2, intercept=False)
regr = regr.fit(X=data_x, y=data_y)
temp = regr.get_factors()
print("R2total", regr.score(X=data_x, y=data_y))
print("R2pred", regr.score(X=data_x, y=data_y, mean_factor=True))
print("R2total_x", regr.score(X=data_x, y=data_y, data_type="portfolio"))
print("R2pred_x", regr.score(X=data_x, y=data_y, mean_factor=True,
                             data_type="portfolio"))
print(datetime.now() - t0)


[========================================================================] 100%


The panel dimensions are:
n_samples: 11 , L: 2 , T: 20


LinAlgError: Array must not contain infs or NaNs.

In [23]:
set(i[0] for i in data.index)

{6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [24]:
for i in set(i[0] for i in data.index):
    data.loc[(i,1935),'invest'] = np.nan
    

In [21]:
temp[1][0].shape

(20,)

In [ ]:
data.set_index(['firm', 'year'])